In [1]:
import pandas as pd
import os
import subprocess
from tqdm import tqdm
import re


In [2]:
class suppress_stdout_stderr(object):
    '''
    A context manager for doing a "deep suppression" of stdout and stderr in
    Python, i.e. will suppress all print, even if the print originates in a
    compiled C/Fortran sub-function.
       This will not suppress raised exceptions, since exceptions are printed
    to stderr just before a script exits, and after the context manager has
    exited (at least, I think that is why it lets exceptions through).

    '''
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])


In [11]:
def extract_leap_info(log_text):
    # 使用正则表达式匹配错误、警告和注意的数量
    match = re.search(r"Exiting LEaP: Errors = (\d+); Warnings = (\d+); Notes = (\d+).", log_text)
    if match:
        errors, warnings, notes = map(int, match.groups())
        return errors, warnings, notes
    else:
        return None
    
def minimize_use_amber(input_pdb_file, output_path, AMBERHOME='/home/xiaoruiwang/software/amber20'):
    is_success = False
    abs_input_pdb_path = os.path.abspath(input_pdb_file)
    abs_output_path = os.path.abspath(output_path)
    pdb_name = os.path.split(abs_input_pdb_path)[-1].split('.')[0]
    minimize_command = f'./minimize_script_folder/minimize_protein.sh {abs_input_pdb_path} {abs_output_path} {AMBERHOME}'
    if not os.path.exists(os.path.join(output_path, f'{pdb_name}_minimized.pdb')):
        completed_process = subprocess.run(minimize_command, shell=True)
        if completed_process.returncode != 0:
            print("Error: The command did not run successfully!")
    
    # 抓错误

    with open(os.path.join(output_path, f'{pdb_name}_minimized_tleap.log'), 'r') as f:
        for line in f.readlines():
            line = line.strip()
            info = extract_leap_info(line)
            if info:
                errors, warnings, notes = info
                if errors == 0:
                    is_success = True

        
    return abs_output_path, is_success

def get_mcsa_normal_dataset(dir, subset=['train', 'valid'], flag='mcsa', read_new=False):
    dataset = pd.DataFrame()
    for dataset_flag in subset:
        sub_df = pd.read_csv(os.path.join(dir, f'{dataset_flag}_dataset' if not read_new else f'new_{dataset_flag}_dataset', f'{flag}_{dataset_flag}.csv'))
        sub_df['dataset_flag'] = [dataset_flag for _ in range(len(sub_df))]
        dataset = pd.concat([dataset, sub_df])
    return dataset
        
    

In [4]:
# aug_dir_flag ='mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123'
# aug_dir_flag = 'mcsa_aug_40_mutation_rate_0.3_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123'
aug_dir_flag = 'mcsa_aug_20+40_mutation_rate_0.2+0.35_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123'
new_aug_save_path = f'../dataset/mcsa_fine_tune/{aug_dir_flag}/'

fixed_generated_structure_dir = f'../dataset/mcsa_fine_tune/{aug_dir_flag}/esmfold_generated_aug_structures'
end_aug_dataset = get_mcsa_normal_dataset(new_aug_save_path, subset=['train', 'valid'], flag='aug_mcsa', read_new=True)
end_aug_dataset

,reaction,ec,alphafolddb-id,aa_sequence,site_labels,site_types,cluster,ec_level1,dataset_flag
0,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593,MSIQHFRVALIPFFAAFCLPVFAHPETLVKVKDAEDQLGARVGYIE...,"[[68], [71], [128], [164], [232], [235]]",NaN,Cluster 18632,3,train
1,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m7,TRPEERFPMMSTMKRLLCGAVLSRWPEWEQLGRRICYEQGTLVESP...,"[[11], [14], [66], [99], [158], [161]]",NaN,Cluster 18632,3,train
2,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m0,SGPPFPMLSTGKVLCGCVLRSMAGKCQLGRVIMYMQNDWYEYSPTE...,[],NaN,Cluster 18632,3,train
3,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m10,FRPEEFPMMSTFKFLLCNAVLSCVDAGQEQLGRVISQILEEQMPVR...,[],NaN,Cluster 18632,3,train
4,CC1C(=O)N(C)C1C.O|MSIQHFRVALIPFFAAFCLPVFAHPETL...,3.5.2.6,P62593-c0_m19,FPEERFPMSLKVLLCGAVLSRVDAGQEEGRRIYYSQDLVEYSVTDK...,[],NaN,Cluster 18632,3,train
...,...,...,...,...,...,...,...,...,...
987,CP(=O)([O-])OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[...,6.1.1.11,Q46AN5-c0_m360,RIGGMCDAQCPCFWMYVGETLPQEVKQFTRSGTSIRYESGIHGLTR...,[],NaN,Cluster 3619,6,valid
988,CP(=O)([O-])OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[...,6.1.1.18,P00962-c0_m50,HTTVHRFQELNGYKHIGVSKSICVNFGRAQDYTYQWECFDTNCKIN...,[],NaN,Cluster 1462,6,valid
989,CP(=O)([O-])OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[...,6.1.1.18,P00962-c0_m60,HETVQDRFPPEFGVLMVGHAMHNWTYIACDYYGMCNLRTDYTNVKE...,[],NaN,Cluster 1462,6,valid
990,NC(=[NH2+])NCCC[C@H]([NH2+]CCC(=O)[O-])C(=O)[O...,6.3.3.4,P0DJQ7-c0_m130,MVEKRINLWKYKADNPLGGVMHREDVRLELDTNGAYDHATFDLNER...,[],NaN,Cluster 2977,6,valid


In [8]:
end_aug_dataset['site_labels'].apply(lambda x:x=='[]').sum()/len(end_aug_dataset)

0.6717156417247299

In [13]:
minimize_failure_list = []

for pdb_id in tqdm(end_aug_dataset['alphafolddb-id'].tolist()):
    pdb_id = pdb_id.replace(' ', '')
    abs_input_path = os.path.abspath(os.path.join(fixed_generated_structure_dir, f'{pdb_id}.pdb'))
    # print(abs_input_path)
    # break
    _, is_success = minimize_use_amber(abs_input_path, os.path.abspath(fixed_generated_structure_dir), AMBERHOME='/home/ipmgpu2022a/Software/amber20')
    if not is_success:
        minimize_failure_list.append(abs_input_path)

 40%|████      | 3964/9903 [00:00<00:00, 9849.00it/s]Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG


Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20+40_mutation_rate_0.2+0.35_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P62593-c0_m180_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DENORMAL


Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20+40_mutation_rate_0.2+0.35_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P62593-c0_m40_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DENORMAL


...

Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DENORMAL


Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20+40_mutation_rate_0.2+0.35_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P0A731-c0_m40_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_DENORMAL
 49%|████▉     | 4868/9903 [02:32<02:38, 31.83it/s]

Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20+40_mutation_rate_0.2+0.35_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P0A731-c0_m380_minimized.pdb


KeyboardInterrupt: 

In [6]:
minimize_failure_list

In [7]:
end_aug_test_dataset = get_mcsa_normal_dataset(new_aug_save_path, subset=['test'], flag='aug_mcsa', read_new=True)
end_aug_test_dataset



,reaction,ec,alphafolddb-id,aa_sequence,site_labels,site_types,cluster,ec_level1,dataset_flag
0,S1[Fe]S[Fe+]1.S1[Fe]S[Fe+]1.[H+].[H+]|MSRTVMER...,1.12.7.2,P07598,MSRTVMERIEYEMHTPDPKADPDKLHFVQIDEAKCIGCDTCSQYCP...,"[[156], [159], [178], [198], [237], [240], [24...",NaN,Cluster 8052,1,test
1,O=O.[Fe+2].[Fe+2].[Fe+2].[Fe+2].[H+].[H+].[H+]...,1.9.3.1,Q5SJ80,MAVRASEISRVYEAYPEKKATLYFLVLGFLALIVGSLFGPFQALNY...,"[[86], [88], [233], [237], [384], [385], [386]...",NaN,Cluster 1144,1,test
2,C.NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)([O-])OP(=O...,1.14.13.25,P27353,MSQPQSSQVTKRGLTDPERAAIIAAAVPDHALDTQRKYHYFIQPRW...,"[[114], [144], [147], [209], [243], [246]]",NaN,Cluster 9876,1,test
3,O=C([O-])c1ccc(O)c(O)c1.O=O|MPIELLPETPSQTAGPYV...,1.13.11.3,P00436,MPIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAP...,"[[109], [148], [158], [161], [163]]",NaN,Cluster 24440,1,test
4,CN[C@@H](CSSC[C@H](NC)C(C)=O)C(C)=O.S1[Fe]S[Fe...,1.8.7.2,Q55389,MTSSDTQNNKTLAAMKNFAEQYAKRTDTYFCSDLSVTAVVIEGLAR...,"[[56], [58], [75], [77], [86], [87], [88]]",NaN,Cluster 27204,1,test
...,...,...,...,...,...,...,...,...,...
90,CP(=O)([O-])OC[C@H]1O[C@@H](n2cnc3c(N)ncnc32)[...,6.1.1.20,P27001,MLEEALAAIQNARDLEELKALKARYLGKKGLLTQEMKGLSALPLEE...,"[[149], [178], [204], [218], [261], [314]]",NaN,Cluster 13284,6,test
91,NC(=O)CC[C@H]([NH3+])C(=O)[O-].Nc1ncnc2c1ncn2[...,6.3.5.4,P22106,MCSIFGVFDIKTDAVELRKKALELSRLMRHRGPDWSGIYASDNAIL...,"[[2], [51], [75], [76], [322], [325]]",NaN,Cluster 1460,6,test
92,C[C@H]1O[C@H](COP(C)(=O)[O-])[C@@H](OP(=O)([O-...,6.5.1.4,P46849,MKRMIALDGAQGEGGGQILRSALSLSMITGQPFTITSIRAGRAKPG...,"[[13], [308]]",NaN,Cluster 14477,6,test
93,C[C@@H](O)C(=O)[O-].C[C@@H]([NH3+])C(=O)[O-].N...,6.1.2.1,P25051,MNRIKVAILFGGCSEEHDVSVKSAIEIAANINKEKYEPLYIGITKS...,"[[19], [22], [99], [244], [250], [290], [311],...",NaN,Cluster 13982,6,test


In [8]:
import shutil

minimize_failure_list_test = []
org_structure_dir = '../dataset/mcsa_fine_tune/structures/alphafolddb_download'
for pdb_id in tqdm(end_aug_test_dataset['alphafolddb-id'].tolist()):
    pdb_id = pdb_id.replace(' ', '')
    org_fname = 'AF-{}-F1-model_v4.pdb'.format(pdb_id.split('_')[0].split('-')[0])
    org_pdb_path = os.path.join(org_structure_dir, org_fname)
    abs_input_path = os.path.abspath(os.path.join(fixed_generated_structure_dir, f'{pdb_id}.pdb'))
    try:
        shutil.copyfile(org_pdb_path, os.path.join(fixed_generated_structure_dir, '{}.pdb'.format(pdb_id.split('_')[0].split('-')[0])))
    except:
        minimize_failure_list_test.append(abs_input_path)
        continue
    
    # print(abs_input_path)
    # break
    _, is_success = minimize_use_amber(abs_input_path, os.path.abspath(fixed_generated_structure_dir), AMBERHOME='/home/ipmgpu2022a/Software/amber20')
    if not is_success:
        minimize_failure_list_test.append(abs_input_path)

  1%|          | 1/95 [00:01<02:24,  1.53s/it]

Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P07598_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_DENORMAL
  4%|▍         | 4/95 [00:02<00:47,  1.91it/s]

Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P00436_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_DENORMAL
  5%|▌         | 5/95 [00:03<00:49,  1.81it/s]

...

Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P22106_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_DENORMAL
 98%|█████████▊| 93/95 [01:58<00:02,  1.40s/it]

Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P46849_minimized.pdb


Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_OVERFLOW_FLAG IEEE_DENORMAL
100%|██████████| 95/95 [01:59<00:00,  1.26s/it]

Minimization complete. Optimized structure is in /raid/xiaoruiwang/works/ec_site_prediction/dataset/mcsa_fine_tune/mcsa_aug_20_mutation_rate_0.2_insertion_rate_0.1_deletion_rate_0.1_max_length_150_seed_123/esmfold_generated_aug_structures/P25051_minimized.pdb


In [9]:
print(len(minimize_failure_list_test))

13
